## 高德key获取方法

1.使用自己的手机号注册，并用身份证激活开发者资格，可获取10个key,每个key有3W额度/day，共30W额度/day。

2.进入“虚拟手机号”相关网站，使用其虚拟手机号注册或登录高德平台，然后激活开发者资格，可使用google搜索身份证照片，在激活页面输入身份证正面的正确信息，身份证背面可随意使用，激活成功后记录key(虚拟手机号过一段时间会被停用，但key仍可用），共30W额度/day。
目前收集到的网站如下：
（1）https://www.cnwml.com/   （2）https://jiemahao.com/

In [4]:
# 高德poi爬虫需要用到的key
aks = ['6f358af5ffc1f4ed7942c0a2f4858f0c', '00d73e3b60cf52dbb6854b7924eb2926', '921310d4850530e5215f6b49bce62e8d', '6e07247bfb1527bbf182775c2e1e6ffe',
          '05c12541663e4e12bb8b3a0a58bc7471', '243f773858f5ec0be8799d13f59d402c', '50e48a9221b9fac0613768b184c0c248', 'dbf5caadb936092ee42964d299bbdb36',
          'fc30acf1460f1969819781ebb9544e3e', 'a389ac4315c9ac144243da956290765e','12d9524a2e6e2a97a75d428159a2ab6c', '36f9d87d6a5d033dacdc647df909f4e5', 'bf11704edcf3602249354aec791d9070',
          '0abe2eb40a0eace8e942447f0a6bb57b', 'c42958826c35068fb0ef6aad8cb80310', '53b63a6cf63693640533fd33066ae901', 'daf7544a8507720c91c1064ee5440e55']

In [1]:
import requests
import json
import time
import pandas as pd
 
# 构建URL访问API部分
class GaoPoi(object):
    def __init__(self, query, loc, city, ak):  # query:行业分类，loc:检索的位置坐标，ak：服务秘钥
        self.query = query
        self.loc = loc
        self.ak = ak
        self.city = city
 
    # 构建访问URL
    def urls(self):
        urls = []
        for i in range(0, 50):
            url = 'http://restapi.amap.com/v3/place/polygon?types=' + self.query + '&polygon=' + str(self.loc) + '&offset=20&page=' + str(i) + '&output=JSON&key=' + str(self.ak)
           # print(url)
            urls.append(url)
        return urls
 
    # 访问APIP获取数据
    def get_data(self):
        for i, url in enumerate(self.urls()):
            try:
                print(url)
                #request.install_opener(opener)
               # js = request.urlopen(targetUrl).text
                js = requests.get(url).text
                data = json.loads(js)
                if data['count'] != '0' and data['status'] != '0':
                    for item in data['pois']:
                        js = {}
                        js['一级行业'] = h1
                        js['二级行业'] = h2
                        js['name'] = item['name']
                        js['address'] = item['address']
                        js['location'] = item['location']
                        js['cityname'] = item['cityname']
                        js['adname'] = item['adname']
                        #js['lng'] = item['location']['lng']
                        yield js
                elif data['status'] != '0':
                    print(url)
                    print('本页及以后无数据！')
                    break
                else:
                    raise RuntimeError('额度用完！')           
                    break
            except:
                print('出现错误！')
                with open('./log.txt', 'a') as fl:
                    fl.write(url+'\n')
                global ak
                global aks
                self.ak = ak = aks.pop()
                self.get_data()
                break
                
    @staticmethod
    def deal_max(dt,Max, grib, ak, loc, city): #处理POI高密度格点
        
        def trans_loc(loc): #loc还原
            return str(loc.split('|')[1]).split(',')[1] + ',' + str(loc.split('|')[1]).split(',')[0] + ',' + str(loc.split('|')[0]).split(',')[1] + ',' + str(loc.split('|')[0]).split(',')[0]
        
        def judge_loc(loc):
            return loc.split('|')[1] == loc.split('|')[0]
        
        if len(dt) > Max and str(city) in dt['cityname'].values:
            loc_new = trans_loc(loc)
            locs_small = LocalDiv(loc_new, grib)
            locs_will_use = locs_small.ls_row()
            for loc in locs_will_use:
                par = GaoPoi(h2, loc, ak)
                dt = par.get_data()
                df = pd.DataFrame(dt)
                if judge_loc(loc) is True:
                    return True
                if GaoPoi.deal_max(df, 899, grib/4, ak, loc, city) is True:
                    continue
                elif len(df) != 0:
                    print(df)
                    df.to_csv(file_name,header=0,index=False,encoding='utf_8_sig',mode='a+')
                    time.sleep(0.5)
                else:
                    pass
            return True
        else:
            return False    


  
  
 
# 大网格划分成小网格部分
class LocalDiv(object):
    def __init__(self, loc_all,
                 divd):  # loc_all：为构建访问url中的左下角（西南）坐标和右上角（东北）坐标（bounds=39.915,116.404,39.975,116.414）；divd：分割网格大小
        self.loc_all = loc_all
        self.divd = divd
 
    # 划分纬度
    def lat_all(self):
        lat_sw = float(self.loc_all.split(',')[0])  # 西南方向纬度
        lat_ne = float(self.loc_all.split(',')[2])  # 东北方向纬度
        lat_list = [str(lat_ne)]
        while lat_ne - lat_sw > 0:
            m = lat_ne - self.divd
            lat_ne = lat_ne - self.divd
            lat_list.append("{:.2f}".format(m))
        return sorted(lat_list)
 
    # 划分经度
    def lng_all(self):
        lng_sw = float(self.loc_all.split(',')[1])  # 西南方向经度
        lng_ne = float(self.loc_all.split(',')[3])  # 东北方向经度
        lng_list = [str(lng_ne)]
        while lng_ne - lng_sw > 0:
            m = lng_ne - self.divd
            lng_ne = lng_ne - self.divd
            lng_list.append("{:.2f}".format(m))
        return sorted(lng_list)
 
    # 将划分的经纬度进行组合
    def ls_com(self):
        lat = self.lat_all()
        lng = self.lng_all()
        latlng_list = []
        for i in range(0, len(lat)):
            a = lat[i]
            for i2 in range(0, len(lng)):
                b = lng[i2]
                ab = b + ',' + a
                latlng_list.append(ab)
        return latlng_list
 
    # 构建每个小网格的西南和东北点的坐标对
    def ls_row(self):
        lat = self.lat_all()
        lng = self.lng_all()
        latlng_list = self.ls_com()
        ls = []
        for n in range(0, len(lat) - 1):
            for i in range(len(lng) * n, len(lng) * (n + 1) - 1):
                coor_a = latlng_list[i]
              #  coor_b = latlng_list[i + 1]
               # coor_c = latlng_list[i + len(lng)]
                coor_d = latlng_list[i + len(lng) + 1]
                coor = coor_d + '|' + coor_a
                #print(coor)
                ls.append(coor)
        return ls      
 
if __name__ == '__main__':
    
    # 行业划分,根据需要可以自己构建行业划分标准
    pois_id = pd.read_excel('/root/poi/map.xlsx')
    
    """
    poi_id = ["140" + str(i) + "00" for i in range(1, 10)] #修改中类参数
    poi_id_add = ["150" + str(i) + "00" for i in range(1, 10)]
    poi_id_add_add = ["141" + str(i) + "00" for i in range(0, 6)]
    poi_id4 = ["151" + str(i) + "00" for i in range(0, 4)]
    pois = { '室内设施': ['970000'], '通行设施': ['990000'] }#
    """
    
    print('----------开始爬取数据！----------')
    start_time = time.time()
    
    grib = 0.06 
    city = '上海' #爬虫城市选择
    locs = LocalDiv('30.65,120.86,31.88,122.2', grib)  # 查询范围坐标,网格大小，现在采取0.06度进行分割
    locs_to_use = locs.ls_row()
    
    ak = aks.pop()


    #for h1, v in pois.items():
    for index, item in pois_id.iterrows():
        h1 = item['大类']
        h2 = item['NEW_TYPE']
        print('爬取：', h1)
        file_name = '/root/poi/gaode_shanghai2.0.csv'.format(h1)
        for loc in locs_to_use:
           #for h2 in v:  # 获取二级行业
            par = GaoPoi(h2, loc, city, ak)
            dt = par.get_data()
            df = pd.DataFrame(dt)
            if par.deal_max(df, 899, grib/2, ak, loc, city) is True:
                continue
            elif len(df) != 0:
                print(df)
                df.to_csv(file_name,header=0,index=False,encoding='utf_8_sig',mode='a+')
                time.sleep(1)
            else:
                pass
    end_time = time.time()
    print('所有poi数据已经爬取完毕，共耗时{:.2f}秒'.format(end_time - start_time))

----------开始爬取数据！----------


NameError: name 'aks' is not defined